### Import

In [79]:
import pandas as pd
import altair as alt
import numpy as np
import os


In [80]:
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

Altair plot

Consideration and Final Visual Target:

I want to understand the daily electrical load and trends of the appliances in my home5 and capture important anomalies, such as power limit days, blackout days, or whether the electricity use of rank3 appliances has changed over time.

A Rationale for Design Decisions:

Candidate images to show time trends are bar charts, scatter charts, and linear charts, which are not suitable because of the long-time span of the time series displayed and the daily granularity of analysis. Because up to 8 types of furniture are displayed, a scatter plot would reduce readability, so it was finally decided to use a line chart with an additional measurement scale to help the reader see the specific value of the electrical load of the furniture on a given day.

In [81]:
def concat_csv(folder_path):
    # get all csv in folder
    csv_list = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    # concat all csv
    df = pd.concat([pd.read_csv(os.path.join(folder_path, csv)) for csv in csv_list], axis=1)
    df.columns = [csv.split('.')[0] for csv in csv_list]
    return df


def get_tidy_data(path, furniture_type):
    df = concat_csv(path)
    data = df.T.sort_index()
    data = data.replace(-1, np.nan)
    data_day = (data.T.groupby(data.T.index // len(data.T.index)).sum()).T
    data_day = data_day.astype(int)
    data_day.reset_index(inplace=True)
    data_day.rename(columns={0: 'overload', 'index': 'date'}, inplace=True)
    data_day['furniture_type'] = furniture_type
    return data_day


furniture = {'01': 'Fridge', '02': 'KitchenAppliances', '03': 'Lamp', '04': 'StereoAndLaptop', '05': 'Freezer',
             '06': 'Tablet', '07': 'Entertainment', '08': 'Microwave'}

In [82]:
df = concat_csv('/Users/shufengli/Desktop/eco/05/01')
data = df.T.sort_index()
data = data.replace(-1, np.nan)
data_day = (data.T.groupby(data.T.index // len(data.T.index)).sum()).T
data_day = data_day.astype(int)
data_day.reset_index(inplace=True)
data_day.rename(columns={0: 'overload', 'index': 'date'}, inplace=True)

In [83]:
df_family5_Fridge = get_tidy_data('/Users/shufengli/Desktop/eco/05/01', furniture['01'])
df_family5_KitchenAppliances = get_tidy_data('/Users/shufengli/Desktop/eco/05/02', furniture['02'])
df_family5_Lamp = get_tidy_data('/Users/shufengli/Desktop/eco/05/03', furniture['03'])
df_family5_StereoAndLaptop = get_tidy_data('/Users/shufengli/Desktop/eco/05/04', furniture['04'])
df_family5_Freezer = get_tidy_data('/Users/shufengli/Desktop/eco/05/05', furniture['05'])
df_family5_Tablet = get_tidy_data('/Users/shufengli/Desktop/eco/05/06', furniture['06'])
df_family5_Entertainment = get_tidy_data('/Users/shufengli/Desktop/eco/05/07', furniture['07'])
df_family5_Microwave = get_tidy_data('/Users/shufengli/Desktop/eco/05/08', furniture['08'])

In [84]:
df_family5 = pd.concat(
    [df_family5_Fridge, df_family5_KitchenAppliances, df_family5_Lamp, df_family5_StereoAndLaptop, df_family5_Freezer,
     df_family5_Tablet, df_family5_Entertainment, df_family5_Microwave], axis=0)

In [85]:
df_family5

,date,overload,furniture_type
0,2012-06-27,500115,Fridge
1,2012-06-28,333726,Fridge
2,2012-06-29,349708,Fridge
3,2012-06-30,393604,Fridge
4,2012-07-01,389406,Fridge
...,...,...,...
20,2012-09-26,4,Microwave
21,2012-09-27,6,Microwave
22,2012-09-28,17,Microwave
23,2012-09-29,2,Microwave


In [86]:
family5_all_furniture = pd.concat(
    [df_family5_Fridge, df_family5_KitchenAppliances, df_family5_Lamp, df_family5_StereoAndLaptop, df_family5_Freezer,
     df_family5_Tablet, df_family5_Entertainment, df_family5_Microwave])
family5_all_furniture.reset_index(inplace=True)
family5_all_furniture.drop(['index'], axis=1, inplace=True)
family5_all_furniture.rename(columns={'variable': 'hours', 'value': 'overload'}, inplace=True)



In [87]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

line = alt.Chart(family5_all_furniture).mark_line(interpolate='basis').encode(
    alt.X('date', title='Date'),
    alt.Y('overload', title='Electricity Overload'),
    color='furniture_type',
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(family5_all_furniture).mark_point().encode(
    x='date',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'overload', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(family5_all_furniture).mark_rule(color='gray').encode(
    x='date',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
all_plot = alt.layer(
    line, selectors, points, rules, text
).properties(
    width=2400, height=800
)


In [100]:
display(all_plot)

alt.LayerChart(...)

Discovery:

Between 2012-9-14 and 2012-9-15, between 2012-9-27 and 2012-9-28, the power load of each furniture drops significantly, so there may be power restrictions on this day.
Between 2012-9-8 and 2012-9-28, the electrical load of the microwave oven was 0, but the electrical load of other furniture existed, so the microwave oven was damaged during this time.
The power load of the refrigerator has been rank1, but the trend of the power load has changed. Before September 2012, the power load of the refrigerator fluctuated around 4,500,000, but after that time period the power load of the refrigerator fluctuated around 3,500,000.
Fridge's electrical load fluctuates widely and on a 5-6 day cycle, requiring further investigation to understand the cause.

In [89]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

Plotly plot

Consideration and Final Visual Target:

I would like to know the electrical appliance usage for each day in Home 4, especially the percentage of electrical load. However, considering the long timeline in the dataset, using one graph to show the 24h electrical load share of household appliances for each day of household 4 would result in too long x states reducing readability. Therefore, I replace the x-axis timeline with the month, and the final visualization goal is the average value of the 24h electricity load share of household 4’s household appliances for each month.

A Rationale for Design Decisions:

My visualization goal needs to include two dimensions: TimeLine and the percentage of electrical load of each household appliance, the candidate diagrams are pie chart, bar chart and stacked bar chart, the pie chart cannot show the trend of electrical load over time, and the bar chart cannot show the percentage of electrical load, the stacked bar chart meets the visualization conditions of both dimensions and is the best choice.



In [90]:
def concat_csv(folder_path):
    # get all csv in folder
    csv_list = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    # concat all csv
    df = pd.concat([pd.read_csv(os.path.join(folder_path, csv)) for csv in csv_list], axis=1)
    df.columns = [csv.split('.')[0] for csv in csv_list]
    return df

In [91]:
def get_tidy_data(path, furniture_type):
    df = concat_csv(path)
    data = df.T.sort_index()
    data = data.replace(-1, np.nan)
    data_24h = (data.T.groupby(data.T.index // 3600).sum()).T
    data_24h.columns = [i for i in range(0, 24)]
    data_24h.index = [i[:7] for i in data_24h.index]
    output = pd.melt(data_24h.reset_index().rename(columns={'index': 'date'}), ['date'])
    output['furniture_type'] = furniture_type
    return output

In [92]:
furniture = {'01': 'Fridge', '02': 'KitchenAppliances', '03': 'Lamp', '04': 'StereoAndLaptop', '05': 'Freezer',
             '06': 'Tablet', '07': 'Entertainment', '08': 'Microwave'}

df_family4_Fridge = get_tidy_data('/Users/shufengli/Desktop/eco/04/01', furniture['01'])
df_family4_KitchenAppliances = get_tidy_data('/Users/shufengli/Desktop/eco/04/02', furniture['02'])
df_family4_Lamp = get_tidy_data('/Users/shufengli/Desktop/eco/04/03', furniture['03'])
df_family4_StereoAndLaptop = get_tidy_data('/Users/shufengli/Desktop/eco/04/04', furniture['04'])
df_family4_Freezer = get_tidy_data('/Users/shufengli/Desktop/eco/04/05', furniture['05'])
df_family4_Tablet = get_tidy_data('/Users/shufengli/Desktop/eco/04/06', furniture['06'])
df_family4_Entertainment = get_tidy_data('/Users/shufengli/Desktop/eco/04/07', furniture['07'])
df_family4_Microwave = get_tidy_data('/Users/shufengli/Desktop/eco/04/08', furniture['08'])

In [93]:
family4_all_furniture = pd.concat(
    [df_family4_Fridge, df_family4_KitchenAppliances, df_family4_Lamp, df_family4_StereoAndLaptop, df_family4_Freezer,
     df_family4_Tablet, df_family4_Entertainment, df_family4_Microwave])

family4_all_furniture.reset_index(inplace=True)
family4_all_furniture.drop(['index'], axis=1, inplace=True)
family4_all_furniture.rename(columns={'variable': 'hours', 'value': 'overload'}, inplace=True)

In [94]:
show_data = family4_all_furniture.groupby(['date', 'hours', 'furniture_type']).mean().reset_index()
show_data.sort_values(by=['hours', 'furniture_type'], inplace=True)

time_stamp = {0: '0h', 1: '1h', 2: '2h', 3: '3h', 4: '4h', 5: '5h', 6: '6h', 7: '7h', 8: '8h', 9: '9h', 10: '10h',
              11: '11h', 12: '12h', 13: '13h', 14: '14h', 15: '15h', 16: '16h', 17: '17h', 18: '18h', 19: '19h',
              20: '20h', 21: '21h', 22: '22h', 23: '23h'}
show_data['hours'] = show_data['hours'].map(time_stamp)

In [95]:
fig = px.bar(show_data, x='hours', y='overload', animation_frame='date', color='furniture_type')
fig.layout.title = 'The mean of electricity overload in Family Fourth'
fig.layout.yaxis.title = 'Electricity Overload'
fig.layout.xaxis.title = 'Hours'
fig.show()

Discovery:

From June to December 2016, refrigerators electricity overload rank 1 of all appliances, but by January 2013, they shrank to 1/7 of their original consumption, guessing that the low winter temperatures reduced refrigerator usage.
Kitchen appliances are found to be in use from 6am-8am and 6pm-7pm.
Microwave were found to be in use from 11am-12am and 6pm.
Because of the relationship between the direct sun point movement, the closer the time is to winter, the longer the time of darkness, the greater the proportion of time and load of light use.
